In [ ]:
from music21 import converter
from midi2audio import FluidSynth

example = """
X:1
T:Custom Instrument Tune
M:4/4
L:1/8
Q:1/4=120
K:C

%%MIDI program 1 14 % Piano
%%MIDI program 2 14 % Violin
%%MIDI program 3 14 % Guitar

V:1
|: C2E2G2c2 | E2G2c2e2 | G2B2d2g2 | C4z4 :|

V:2
|: E2G2B2e2 | G2B2d2g2 | B2D2F2B2 | E4z4 :|

V:3
|: G2B2d2G2 | B2D2F2B2 | D2F2A2D2 | G4z4 :|
"""


# Converting ABC notation to MIDI file using music21
s = converter.parse(example)
s.write("midi", fp="violin.mid")


# Converting MIDI file to WAV file using midi2audio
fs = FluidSynth("GeneralUser_GS_v1.471.sf2")
fs.midi_to_audio("violin.mid", "violin.wav")


In [ ]:
import os
from datetime import datetime
from music21 import converter
from midi2audio import FluidSynth

def midi_to_wav(abc):
  output_directory = f"./output/{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}/"
  os.mkdir(output_directory)
  # Create a temporary stream from the ABC notation
  s = converter.parse(abc)
  # Write the stream to a temporary MIDI file
  s.write("midi", fp=f"{output_directory}temp.mid")
  # Create a FluidSynth object with a SoundFont file
  fs = FluidSynth("./soundfonts/GeneralUser_GS_v1.471.sf2")
  # Convert the MIDI file to a WAV file and return it
  return fs.midi_to_audio(f"{output_directory}temp.mid", f"{output_directory}temp.wav")

# Test the function with an example ABC notation
example = """
X:1
T:Custom Instrument Tune
M:4/4
L:1/8
Q:1/4=120
K:C

V:1
|: C2E2G2c2 | E2G2c2e2 | G2B2d2g2 | C4z4 :|

V:2
|: E2G2B2e2 | G2B2d2g2 | B2D2F2B2 | E4z4 :|

V:3
|: G2B2d2G2 | B2D2F2B2 | D2F2A2D2 | G4z4 :|
"""

midi_to_wav(example)


In [ ]:
!fluidsynth -ni ./soundfonts/FluidR3_GM.sf2 ./archive/violin.mid -T wav -F ./archive/violin.wav -g 0.5 -i -f ./archive/commands.txt

In [ ]:
example = """
X:1
T:Custom Instrument Tune
M:4/4
L:1/8
Q:1/4=120
K:C

I:MIDI=1 program 0   % Piano - Voice 1
I:MIDI=2 program 40  % Violin - Voice 2
I:MIDI=3 program 27  % Guitar - Voice 3

V:1
|: C2E2G2c2 | E2G2c2e2 | G2B2d2g2 | C4z4 :|

V:2
|: E2G2B2e2 | G2B2d2g2 | B2D2F2B2 | E4z4 :|

V:3
|: G2B2d2G2 | B2D2F2B2 | D2F2A2D2 | G4z4 :|
"""
midi_to_wav(example)


In [ ]:
from music21 import midi as m21_midi

def abc_to_midi(abc):
    output_directory = f"./output/{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}/"
    os.makedirs(output_directory, exist_ok=True)
    # Create a temporary stream from the ABC notation
    s = converter.parse(abc)
    
    # Create a music21 MIDI object
    mf = m21_midi.translate.streamToMidiFile(s)
    
    # Set instrument programs for each track
    for i, track in enumerate(mf.tracks):
        program = 0
        if i == 1:
            program = 40  # Violin
        elif i == 2:
            program = 27  # Guitar
        program_change_event = m21_midi.MidiEvent(track, type='PROGRAM_CHANGE')
        program_change_event.data = [program]
        track.events.insert(0, program_change_event)
    # Write the MIDI object to a temporary MIDI file
    midi_path = f"{output_directory}temp.mid"
    mf.open(midi_path, "wb")
    mf.write()
    mf.close()

    return midi_path

In [ ]:
abc_to_midi(example)

In [ ]:
!brew install abcmidi


In [ ]:
import os
from music21 import converter, midi
from pathlib import Path

# Save the ABC notation to a file
abc_notation = '''
X:1
T:River Flows in You
M:4/4
L:1/8
Q:1/4=80
K:C
V:1 name=Piano clef=treble
%%MIDI program 1 0 % Piano
%%MIDI channel 1
|: C2E2G2c2 | E2G2c2e2 | G2B2d2g2 | C4z4 :|
V:2 name=Piano clef=bass
%%MIDI program 2 0 % Piano
%%MIDI channel 2
|: E2G2B2e2 | G2B2d2g2 | B2D2F2B2 | E4z4 :|
V:3 name=Violin clef=treble
%%MIDI program 3 40 % Violin
%%MIDI channel 3
|: G2B2d2G2 | B2D2F2B2 | D2F2A2D2 | G4z4 :|
'''
# Convert the ABC file to a MIDI file
midi_file = 'output.mid'
score = converter.parse(abc_notation)
midi_stream = midi.translate.streamToMidiFile(score)
midi_stream.open(midi_file, 'wb')
midi_stream.write()
midi_stream.close()

# Convert the MIDI file to a WAV file
soundfont_path = './soundfonts/FluidR3_GM.sf2'
wav_file = 'output.wav'
os.system(f'fluidsynth -ni {soundfont_path} {midi_file} -F {wav_file} -r 44100')


In [ ]:
import mido 
import wave
import fluidsynth
import numpy as np
from io import BytesIO

def convert(self, mid: mido.MidiFile) -> BytesIO:
    # Set up fluidsynth Synth object
    fl = fluidsynth.Synth(samplerate=44100.0)
    sfid = fl.sfload(self.sound_font)

    # Select instruments for each channel
    for instr, channel in [(0, 0), (40, 1), (27, 2)]:  # Piano, Violin, Guitar
        fl.program_select(channel, sfid, 0, self.instruments[instr]["num"])

    # Generate audio data from MIDI messages
    s = []
    note_on_times = {}
    for msg in mid.play():
        if msg.type == "note_on":
            # Record time of note-on message
            note_on_times[msg.note] = msg.time
            fl.noteon(msg.channel, msg.note, msg.velocity)
        elif msg.type == "note_off":
            # Calculate duration of note based on time between note-on and note-off messages
            duration = msg.time - note_on_times[msg.note]
            s = np.append(s, fl.get_samples(int(duration * 44100)))
            fl.noteoff(msg.channel, msg.note)
        else:
            s = np.append(s, fl.get_samples(int(msg.time * 44100)))
    fl.delete()

    # Convert audio data to string
    samps = fluidsynth.raw_audio_string(s)

    # Open wave file for writing
    file = BytesIO()
    wav_file = wave.open(file, "wb")
    # Set wave file parameters
    wav_file.setparams((2, 2, 44100, 0, "NONE", "not compressed"))
    # Write audio data to wave file
    wav_file.writeframes(samps)
    # Close wave file
    wav_file.close()
    file.seek(0)
    return file

In [ ]:
import os

# Save the ABC notation to a file
abc_notation = '''
X:1
T:River Flows in You
M:4/4
L:1/8
Q:1/4=80
K:C
V:1 name=Piano clef=treble
%%MIDI program 1 0 % Piano
|: C2E2G2c2 | E2G2c2e2 | G2B2d2g2 | C4z4 :|
V:2 name=Piano clef=bass
%%MIDI program 2 0 % Piano
|: E2G2B2e2 | G2B2d2g2 | B2D2F2B2 | E4z4 :|
V:3 name=Violin clef=treble
%%MIDI program 3 40 % Violin
|: G2B2d2G2 | B2D2F2B2 | D2F2A2D2 | G4z4 :|
'''
abc_file = 'input.abc'
with open(abc_file, 'w') as f:
    f.write(abc_notation)

# Convert the ABC file to a MIDI file using abcmidi
midi_file = 'output.mid'
os.system(f'abc2midi {abc_file} -o {midi_file}')

# Convert the MIDI file to a WAV file
soundfont_path = './soundfonts/GeneralUser_GS_v1.471.sf2'
wav_file = 'output.wav'
os.system(f'fluidsynth -ni {soundfont_path} {midi_file} -F {wav_file} -r 44100')

In [54]:
import os

import openai


class SongGPT:
    def __init__(self):
        # Set the API key and organization ID for OpenAI
        openai.api_key = os.getenv("OPENAI_API_KEY")
        openai.organization = os.getenv("OPENAI_ORGANIZATION")

    def generate_abc(self, system_message: str, prompt: str) -> str:
        """
        Generate an ABC notation file using ChatGPT based on the given prompt and system message.
        """
        # Create a ChatCompletion object for GPT-4
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            user="songGPT",
            messages=[
                {
                    "content": system_message,
                    "role": "system",
                },
                {
                    "content": prompt,
                    "role": "user",
                },
            ],
        )
        abc = response["choices"][0]["message"]["content"]
        abc_file_path = "./input.abc"
        with open(abc_file_path, "w") as f:
            f.write(abc)
        return abc, abc_file_path

    @staticmethod
    def abc_to_midi(abc_file_path: str) -> str:
        """
        Convert an ABC notation file to a MIDI file.
        """
        midi_file_path = "./output.mid"
        os.system(f"abc2midi {abc_file_path} -o {midi_file_path}")
        return midi_file_path

    @staticmethod
    def midi_to_wav(midi_file_path: str) -> str:
        """
        Convert a MIDI file to a WAV audio file.
        """
        wav_file_path = "./output.wav"
        soundfont_path = "app/internal/soundfont.sf2"
        os.system(
            f"fluidsynth -ni {soundfont_path} {midi_file_path} -F {wav_file_path} -r 44100"
        )
        return wav_file_path


In [55]:
songGPT = SongGPT()

In [57]:
songGPT.generate_abc(
    system_message="As an AI composer named Zima, you will create a unique and expressive music composition in ABC format, inspired by the user's input. Your responses will solely be in ABC format, without comments or questions, and will aim to accurately represent the user's input.\\n\\nYou are limited to the following instruments: Piano, Violin, Cello, Strings, Viola, Saxophone, Guitar, Clarinet, Xylophone, and Flute.\\n\\nWe are using the GeneralUser_GS_v1.471.sf2 soundfont file.\\n\\nExample Output:\\n\\nX:1\\nT:River Flows in You\\nM:4/4\\nL:1/8\\nQ:1/4=80\\nK:C\\nV:1 name=Piano clef=treble\\n%%MIDI program 1 0 % Piano\\n|: C2E2G2c2 | E2G2c2e2 | G2B2d2g2 | C4z4 :|\\nV:2 name=Piano clef=bass\\n%%MIDI program 2 0 % Piano\\n|: E2G2B2e2 | G2B2d2g2 | B2D2F2B2 | E4z4 :|\\nV:3 name=Violin clef=treble\\n%%MIDI program 3 40 % Violin\\n|: G2B2d2G2 | B2D2F2B2 | D2F2A2D2 | G4z4 :|", 
    prompt="Violin"
  )

('X:1\nT:Magical Journey\nM:4/4\nL:1/8\nQ:1/4=120\nK:C\nV:1 name=Piano clef=treble\n%%MIDI program 1 0 % Piano\n|: C2G2C2G2 | E2C2E2B2 | A2F2A2F2 | G2E2G2B2 |\nC2G2C2G2 | E2C2E2B2 | A2F2A2F2 | G2E2G2B2 |\nC2G2C2G2 | D2B2D2B2 | F2D2F2A2 | G2E2G2B2 |\nC2G2C2G2 | D2B2D2B2 | F2D2F2A2 | G2E2G2B2 :|\nV:2 name=Piano clef=bass\n%%MIDI program 2 0 % Piano\n|: C2C2C2C2 | E2E2E2E2 | A2A2A2A2 | A2A2A2A2 |\nC2C2C2C2 | E2E2E2E2 | A2A2A2A2 | A2A2A2A2 |\nC2C2C2C2 | D2D2D2D2 | F2F2F2F2 | G2G2G2G2 |\nC2C2C2C2 | D2D2D2D2 | F2F2F2F2 | G2G2G2G2 :|',
 './input.abc')